# 환경설정

In [1]:
!pip install konlpy

In [2]:
import re
import pandas as pd
import numpy as np
import itertools
from collections import Counter

### 버전확인

In [3]:
import warnings
warnings.simplefilter("ignore")

import konlpy
konlpy.__version__

'0.6.0'

### KoNlp 설정
- Mecab은 충돌나기 때문에 주석처리했음

- nouns : 명사 추출

- morphs : 형태소 추출

- pos : 품사 부착

In [4]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
# mecab = Mecab()
okt = Okt()

# 토큰화

### 보존할 형태소 리스트

[Kkma 형태소 분류표](https://www.notion.so/caa99ad4954642528b29c6ccd07965f5?v=154c84d4971c4efd83da1f78bb8602d1)

In [54]:
pos_save_lst = [
    'NNG',
    'NNP',
    'NNM',
    'NR',
    'NP',
    'VV',
    'VA',
    'EFQ',
    'EFO',
    'EFA',
    'EFI',
    'EFR',
    'ETN',
    'MAG',
    'XPN',
    'XPV',
    'XSN',
    'XSV',
    'XSA',
    'XSM',
    'XSO',
    'XR',
    'UN'
    ]

### OKT 품사 및 형태소

In [ ]:
okt.tagset

### Kkma 사전 추가 방법
- 아래 경로에 있는 형태소.dic 파일을 수정하면 된다.
- C:\Users\max_s\anaconda3\Lib\site-packages\konlpy\java\dic
- 추가한 이후에는 세션을 재시작해야한다.
- 만약 재시작했는데도 추가가 되지 않는 것 같다면 사용하는 언어 버전이 추가한 언어타입인지 확인해야한다.

In [18]:
f = open("C:\Users\max_s\anaconda3\Lib\site-packages\konlpy\java\dic\noun.dic", "a")
f.write("인분"+"/NNG\n")
f.close()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_127324/1286170530.py, line 1)

### 샘플 텍스트

In [94]:
sampleTextList=[
    '직원분들도 친절하시고 맛 좋음... 양대창 3인분 먹고 느끼해서 김치우동 하나 시켜먹었더니 개운... 짱 맛있음 ㅠㅠ 후식 소프트아이스크림까지 완벽^^~',
    '상암동 맛집 가성비 최고 식사 안주 모두해결됩니다. 안드셔보셨다면 꼭 가보세요 강추',
    '365일 할로윈 컨셉의 레스토랑이라는 점에서 한번 정도는 방문할 만 합니다. 테이블이 다 차지 않았지만 주방이 바쁘다는 이유로 음식이 늦게 나옵니다. 음식 맛은 보통 수준 이상을 기대하기 어렵습니다. 사진속의 리코타 샐러드는 리코타 치즈가 크림치즈보다도 너무 뻑뻑해서 야채와 어울리지 않았습니다. 피자 도우는 페스츄리 질감이었는데, 버터의 향이 강해서 피자치즈를 압도합니다. 먹을 때 가루도 많이 떨어집니다.',
    '개운한 해장국 맛있네요. 친절한 직원들 덕분에 만족스러워요. 먹음직스럽다. 메뉴가 새롭다. 원두가 향기로워요. 건강에 해로울 거 같아요 그릇이 커다랗다',
    '친구들이랑 가봤는대, 아이들이랑 가기 좋아요. 가족들이랑 가세요. 동료들이랑 회식하기 좋아요. 직원들이랑 친목다지니 좋아요 직장인들이 많아요.',
    '원래는 신림 혼맛스시를 가려고 했는데 마침 리모델링 중이라, 전에 맛있게 먹은 기억이 있는 구루메치킨을 왔습니당 오늘의 사시미 퀄리티 여전히 훌륭하고 갈치속젓라구파스타 역시 최고예요 ;ㅅ;회는 2인분정도 시켰는데 같이 나온 전복 진짜 개맛있었어요.. 내장소스 충격적으로 맛있음 사케는 역시 해산물과 먹어야 제맛인 것 같아요!\n\n우니랑 단새우까지 추가로 시켜 먹었더니 초당옥수수를 서비스로 주셨어요우니,단새우 시키면 머리는 튀겨서 따로 나오는데 뭔가 1+1 같고 기분 좋았음 ㅋㅋ 맛도 있었구용!\n우니, 단새우와 같이 나온 오이는 같이 먹는 거라 설명 들었는데 따로 먹는 게 더 나은 것 같아서 걍 피클처럼 먹었음..\n\n창가에 앉았는데 골목뷰라 이쁘진 않았지만 자연광 들어서 사진 잘 나옴!',
    '파스타 종종 가는 곳입니다만 주문이 누락되어서 30분은 기다렸습니다.\n문제는 직원들이 저랑 눈이 자주마주치고 30분가량된 손님이 식사를 안하고 있으면 와서 주문 여부를 물어봐야 하는게 기본이라 보는데 눈 미주쳐도 멀뚱멀뚱쳐다만 보시기에 메뉴 안나오냐 했더니 주문 누락되었다 하네요\n전 입맛이 확 떨어져서 30분 동안 사람 앉혀놓고 뭐하는거냐 버럭 하고 나왔습니다.\n구지 거기서 먹고 싶지는 않더라고요.\n그래서 저는 완전 비추천 합니다',
    '탕수육과 짜장면 양도 많고 맛도 있고 무료배달까지 항상 짜장면 먹을 때 짬뽕 국물이 먹고 싶은데 여기는 1000원 추가 짬뽕국물 먹을 수 있는 추가 메뉴가 좋네요 양도 꽤 되고 뜨근하게 맛있게 잘 먹었습니다.',# 봉평성 id**님의 리뷰 
    '매운 족발이 엄청 맛있었고 양도 많네요. 서비스 막국수는 양 놀랐어요 서비스 수준이 아니라 메인처럼 와서 엄청 배불렀어요'
    ]

### 이모티콘 제거
- Kkma 이모티콘 인식시 에러가 난다

In [65]:
# 파생접미사(동사,형용사)
derivedSuffix = ['XSA','XSV']

# 일반명사, 고유명사, 수사, 대명사, 동사, 형용사, 어근
spacingPos =['NNG', 'NNP', 'NR', 'NP', 'VV', 'VA','XR'] 

In [104]:
# 보존할 형태소
pos_save_lst = [
    'NNG','NNP','NNM','NR','NP',
    'VV','VA','EFQ','EFO','EFA',
    'EFI','EFR','ETN','VCN','ECD',
    'MAG','XPN','XPV','XSN','XSV',
    'XSA','XSM','XSO','XR','UN'
    ]

# 파생접미사(동사,형용사)
derivedSuffix = ['XSA','XSV']

# 일반명사, 고유명사, 수사, 대명사, 동사, 형용사, 어근
spacingPos =['NNG', 'NNP', 'NR', 'NP', 'VV', 'VA','XR','MAG'] 

def deEmojify(text): #이모티콘 제거 함수
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1A0-\U0001F9FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def tokenizer(text): #평서형처리 토큰화
    text = deEmojify(text) #이모티콘 제거
    
    pos_tmp1 = kkma.pos(text)

    lst_tmp1 = []
    for i in pos_tmp1:
        if i[1] in pos_save_lst: # 저장할 형태소만 고르는 조건
            if i[1] in derivedSuffix: #접미사 처리
                lst_tmp1.append((i[0]+'다@',i[1])) #평서형처리를 위한 '다'를 붙인다

            elif i[1] in ['VV','VA']:
                lst_tmp1.append((i[0]+"다@",i[1])) #평서형처리를 위한 '다'를 붙인다
            else:
                lst_tmp1.append(i) # 수정할 필요 없는 것은 그냥 추가
    lst_tmp2 = []
    for i in range(len(lst_tmp1)):
        tmp = lst_tmp1[i]
        if tmp[1] in set(pos_save_lst)-set(spacingPos): #접미사는 띄어쓰지 않는다
            lst_tmp2.append(tmp[0])
        elif tmp[1] in spacingPos: # 단어와 단어 사이는 띄어쓴다
            lst_tmp2.append(" "+tmp[0]) #나머지는 앞부분을 띄어준다
    return "".join(lst_tmp2).split('@')[:-1]

In [86]:
set(pos_save_lst)-set(spacingPos)

{'EFA',
 'EFI',
 'EFO',
 'EFQ',
 'EFR',
 'ETN',
 'MAG',
 'NNM',
 'UN',
 'XPN',
 'XPV',
 'XSA',
 'XSM',
 'XSN',
 'XSO',
 'XSV'}

In [96]:
tmp = deEmojify(sampleTextList[-1])
tmp

'매운 족발이 엄청 맛있었고 양도 많네요. 서비스 막국수는 양 놀랐어요 서비스 수준이 아니라 메인처럼 와서 엄청 배불렀어요'

In [97]:
kkma.pos(tmp)

[('맵', 'VA'),
 ('ㄴ', 'ETD'),
 ('족발', 'NNG'),
 ('이', 'JKS'),
 ('엄청', 'MAG'),
 ('맛있', 'VA'),
 ('었', 'EPT'),
 ('고', 'ECE'),
 ('양도', 'NNG'),
 ('많', 'VA'),
 ('네요', 'EFN'),
 ('.', 'SF'),
 ('서비스', 'NNG'),
 ('막국수', 'NNG'),
 ('는', 'JX'),
 ('양', 'NNG'),
 ('놀라', 'VV'),
 ('었', 'EPT'),
 ('어요', 'EFN'),
 ('서비스', 'NNG'),
 ('수준', 'NNG'),
 ('이', 'JKC'),
 ('아니', 'VCN'),
 ('라', 'ECD'),
 ('메인', 'NNG'),
 ('처럼', 'JKM'),
 ('오', 'VV'),
 ('아서', 'ECD'),
 ('엄청', 'MAG'),
 ('배부르', 'VA'),
 ('었', 'EPT'),
 ('어요', 'EFN')]

In [105]:
# kkma.pos(test_txt0)
tmp = tokenizer(sampleTextList[-1])
print(sampleTextList[-1])
tmp

매운 족발이 엄청 맛있었고 양도 많네요. 서비스 막국수는 양 놀랐어요 서비스 수준이 아니라 메인처럼 와서 엄청 배불렀어요


[' 맵다',
 ' 족발 엄청 맛있다',
 ' 양도 많다',
 ' 서비스 막국수 양 놀라다',
 ' 서비스 수준아니라 메인 오다',
 '아서 엄청 배부르다']